<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pyg/pointnet-classification/02_pointnet_plus_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{pyg-pointnet2-train} -->

# PointNet++ Model 



## Imports


In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

import random
from glob import glob           # ← re-added, in case you use it later
from tqdm.auto import tqdm
 
import wandb

import torch.nn.functional as F

import torch_geometric.transforms as T
from dataset import GainRegressionDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import PointConv
from torch_geometric.nn import MLP, fps, global_max_pool, radius


2.5.1


## Initialize Weights & Biases

We need to call [`wandb.init()`](https://docs.wandb.ai/ref/python/init) once at the beginning of our program to initialize a new job. This creates a new run in W&B and launches a background process to sync data.

In [2]:
os.environ["WANDB_API_KEY"] = "5aac7197ad74c621916ca32f24c2bed880a1d6a6"
# --- 1) WandB initialization --------------------------------------------
wandb_project   = "point_net_traj_planning"               #@param {"type": "string"}
wandb_run_name  = "final-experiment/gain-regression"  #@param {"type": "string"}

wandb.init(
    project=wandb_project,
    name=wandb_run_name,
    job_type="baseline-train"
)

# --- 2) Fill wandb.config -----------------------------------------------
config = wandb.config

# (a) Seed & reproducibility
config.seed = 4242  #@param {type:"number"}
random.seed(config.seed)
torch.manual_seed(config.seed)

# (b) Dataset parameters (replace ModelNet)
#     Point to the folder where your PLYs + labels.csv + opt_successfull.yaml live
config.data_root     = "/home/geriatronics/pmaf_ws/src/dataset_generator/data"  #@param {"type":"string"}
config.labels_csv    = "labels.csv"                                        #@param {"type":"string"}
config.success_yaml  = "opt_successfull.yaml"                         #@param {"type":"string"}

#     Number of points to sample from each cloud
config.npoints       = 2500  #@param {"type":"slider", min:256, max:4096, step:16}

# (c) Split / loader settings
config.test_size     = 0.20  #@param {"type":"slider", min:0.0, max:0.5, step:0.05}
config.batch_size    = 16    #@param {"type":"slider", min:4, max:128, step:4}
config.num_workers   = 0     #@param {"type":"slider", min:1, max:16, step:1}

# (d) Device
config.device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(config.device)

# (e) Model/hyperparameters (you can keep these or adjust)
config.set_abstraction_ratio_1 = 0.748   #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.set_abstraction_radius_1 = 0.4817 #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.set_abstraction_ratio_2 = 0.3316   #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.set_abstraction_radius_2 = 0.2447 #@param {"type":"slider", min:0.1, max:1.0, step:0.01}
config.dropout                 = 0.1    #@param {"type":"slider", min:0.0, max:0.5, step:0.05}

# (f) Optimizer settings
config.learning_rate           = 1e-4  #@param {"type":"number"}
config.epochs                  = 10    #@param {"type":"slider", min:1, max:100, step:1}
config.num_visualization_samples = 20  #@param {"type":"slider", min:1, max:100, step:1}


wandb: Currently logged in as: mateus-salomao4 (mateus-salomao4-technical-university-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Load Dataset

We now load, preprocess and batch the dataset for training, validation/testing and visualization.

In [3]:
# Remove pre_transform and transform since GainRegressionDataset handles sampling & normalization internally

train_dataset = GainRegressionDataset(
    root         = config.data_root,
    labels_csv   = config.labels_csv,
    success_yaml = config.success_yaml,
    split        = "train",
    test_size    = config.test_size,
    random_state = config.seed,
    npoints      = config.npoints,
    augment      = True
)
val_dataset = GainRegressionDataset(
    root         = config.data_root,
    labels_csv   = config.labels_csv,
    success_yaml = config.success_yaml,
    split        = "val",
    test_size    = config.test_size,
    random_state = config.seed,
    npoints      = config.npoints,
    augment      = False
)

train_loader = DataLoader(
    train_dataset,
    batch_size   = config.batch_size,
    shuffle      = True,
    num_workers  = config.num_workers
)
val_loader = DataLoader(
    val_dataset,
    batch_size   = config.batch_size,
    shuffle      = False,
    num_workers  = config.num_workers
)

print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

Train samples: 53, Val samples: 14


## Implementing the PointNet++ Model using PyTorch Geometric

In [4]:
class SetAbstraction(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointConv(nn, add_self_loops=False)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch

In [5]:
class GlobalSetAbstraction(torch.nn.Module):        #MRG strategy -  layer output = PointNet(input) concat with Sampling+Grouping(input)
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch

In [6]:
class PointNet2(torch.nn.Module):
    def __init__(
        self,
        set_abstraction_ratio_1, set_abstraction_ratio_2,
        set_abstraction_radius_1, set_abstraction_radius_2, dropout
    ):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SetAbstraction(
            set_abstraction_ratio_1,
            set_abstraction_radius_1,
            MLP([3, 64, 64, 128])
        )
        self.sa2_module = SetAbstraction(
            set_abstraction_ratio_2,
            set_abstraction_radius_2,
            MLP([128 + 3, 128, 128, 256])
        )
        self.sa3_module = GlobalSetAbstraction(MLP([256 + 3, 256, 512, 1024]))

        self.mlp = MLP([1024, 512, 256, 36], dropout=dropout, norm=None)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)
        x, pos, batch = sa3_out

        return self.mlp(x)

## Training PointNet++ and Logging Metrics on Weights & Biases

In [7]:
# Define PointNet++ model.
model = PointNet2(
    config.set_abstraction_ratio_1,
    config.set_abstraction_ratio_2,
    config.set_abstraction_radius_1,
    config.set_abstraction_radius_2,
    config.dropout
).to(device)

# Define Optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=config.learning_rate
)

In [8]:

def train_step(epoch):
    """Training Step for Regression"""
    model.train()
    epoch_loss = 0.0
    num_batches = len(train_loader)

    progress_bar = tqdm(
        range(num_batches),
        desc=f"Training Epoch {epoch}/{config.epochs}"
    )
    data_iter = iter(train_loader)
    for _ in progress_bar:
        data = next(data_iter).to(device)
        # data.pos: [batch_size, npoints, 3]
        # data.y  : [batch_size, 36] (regression targets)

        optimizer.zero_grad()
        prediction = model(data)                   # [batch_size, 36]
        #print(f"Prediction shape: {prediction.shape}")
        #print(f"Label shape: {data.y.shape}")
        loss = F.mse_loss(prediction, data.y)       # Mean‐squared error
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / num_batches
    wandb.log({
        "Train/Loss": avg_loss,
        "Epoch": epoch
    })


def val_step(epoch):
    """Validation Step for Regression"""
    model.eval()
    epoch_loss = 0.0
    num_batches = len(val_loader)

    progress_bar = tqdm(
        range(num_batches),
        desc=f"Validation Epoch {epoch}/{config.epochs}"
    )
    data_iter = iter(val_loader)
    for _ in progress_bar:
        data = next(data_iter).to(device)
        with torch.no_grad():
            prediction = model(data)
            loss = F.mse_loss(prediction, data.y)
        epoch_loss += loss.item()

    avg_loss = epoch_loss / num_batches
    wandb.log({
        "Validation/Loss": avg_loss,
        "Epoch": epoch
    })





def save_checkpoint(epoch):
    """Save model + optimizer state as a W&B Artifact"""
    ckpt_path = f"checkpoint_epoch_{epoch}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, ckpt_path)

    artifact_name = wandb.util.make_artifact_name_safe(
        f"{wandb.run.name}-{wandb.run.id}-checkpoint"
    )
    ckpt_artifact = wandb.Artifact(artifact_name, type="checkpoint")
    ckpt_artifact.add_file(ckpt_path)
    wandb.log_artifact(ckpt_artifact, aliases=["latest", f"epoch-{epoch}"])



In [9]:
# Prepare an empty W&B Table for visualization
viz_table = wandb.Table(
    columns=[
        "Epoch",
        "PointCloud",
        "Prediction",
        "GroundTruth",
        "MSE_Error"
    ]
)

for epoch in range(1, config.epochs + 1):
    train_step(epoch)
    val_step(epoch)
   # visualize_evaluation(viz_table, epoch)
    save_checkpoint(epoch)

wandb.finish()

Training Epoch 1/10:   0%|          | 0/4 [00:00<?, ?it/s]

Scene: scene_114
Labels: 
 [0.5909635 1.7347713 1.950219  1.0234367 4.1894245 4.7667255 1.3568804
 3.6828344 4.955513  3.1249297 4.1395845 4.79265   4.6581154 3.3271286
 1.1908484 2.4740236 1.8608358 2.7415094 2.9097023 4.929576  1.3971074
 2.9676864 4.3701625 3.1415083 1.0646372 1.0316397 3.7710605 1.7611265
 1.0341299 2.3510613 2.041325  4.6485996 4.656338  1.3037342 3.9204893
 4.813611 ]
Scene: scene_138
Labels: 
 [0.57630014 1.6334505  2.257491   3.6822066  1.2616955  2.8774936
 1.3553008  3.8213751  1.8397629  3.1592948  1.7029408  1.5546244
 1.6375513  1.9945232  2.6053352  1.7500871  3.3291488  1.4180185
 2.146254   4.784334   1.4548609  1.5509473  3.4503753  2.5988064
 4.211947   1.647512   1.8645682  1.4362823  2.725942   1.4183388
 3.2348256  3.9835768  3.5946195  2.588871   1.8565115  1.2364894 ]
Scene: scene_160
Labels: 
 [0.43745694 3.1426024  4.827135   3.253424   4.80379    2.7881312
 1.2428081  2.9546282  3.3386283  1.576006   1.5796162  4.718112
 4.760437   1.8606334  

Validation Epoch 1/10:   0%|          | 0/1 [00:00<?, ?it/s]

Scene: scene_110
Labels: 
 [0.59286314 3.321368   3.9872289  3.7473733  3.4749126  4.903662
 4.6076612  1.6477798  1.4583116  1.898902   1.6272713  4.8462505
 1.4568348  3.3956892  2.0783694  1.454493   2.1345558  2.337087
 2.4154673  4.7286296  4.644076   4.6215067  3.1166477  4.2819877
 3.5449996  4.6295996  4.571115   3.746971   4.557249   3.7359092
 3.1555998  4.8359694  1.5050168  2.0583696  3.202555   3.9031243 ]
Scene: scene_173
Labels: 
 [0.4074984 1.7756864 4.3685474 2.6684132 3.7790463 1.6306722 1.9306862
 2.971937  2.9212248 1.8476715 2.7238503 1.7631072 2.2440672 1.3521923
 3.6121807 1.8163724 1.8638592 2.1998413 4.2758904 1.7675308 2.4085472
 2.049207  1.1624063 2.472029  4.083626  1.2338647 2.2577035 1.2989473
 2.195868  2.8418667 1.6638728 4.673577  2.774746  1.4322546 1.2344766
 3.7412853]
Scene: scene_195
Labels: 
 [0.4346917 4.0873685 4.1074567 4.0546565 4.668514  4.1278005 4.199916
 4.1992664 3.9021645 1.6387109 1.9749628 1.9428326 3.757982  3.851896
 2.0886908 2.721

Training Epoch 2/10:   0%|          | 0/4 [00:00<?, ?it/s]

...

Training Epoch 10/10:   0%|          | 0/4 [00:00<?, ?it/s]

Scene: scene_64
Labels: 
 [0.6903007 2.8889298 4.9581866 4.553757  1.8408002 4.1555176 4.709916
 4.2039814 1.8777328 4.731323  2.514043  3.1843896 3.031479  4.3050184
 3.3786616 3.764748  3.2419045 4.2943673 4.7048163 4.9429817 3.1235223
 1.5006043 2.9039886 4.132203  2.3438318 3.2777352 4.53726   1.1283987
 4.670831  4.9390426 4.671827  4.2960773 4.070412  2.388933  2.2180767
 4.8836436]
Scene: scene_25
Labels: 
 [0.56308967 2.685796   4.7774286  3.1199453  2.233537   3.265739
 4.2911305  1.5637771  2.6840334  1.1467865  3.1962717  4.058394
 2.711791   1.1788917  4.1969643  1.9448647  4.0189047  2.2557151
 3.6268663  1.2078291  4.2057166  2.0817285  1.323597   1.1961863
 1.0670016  2.4786265  2.8887506  2.2878625  2.5361264  3.4977663
 2.1896584  1.4605526  2.9046378  1.6298529  1.9702873  4.4700613 ]
Scene: scene_92
Labels: 
 [0.6180366 1.5553908 3.5650747 2.358486  1.181275  3.6841595 2.999847
 2.5149927 1.6772597 4.5309362 2.8479805 2.7332945 3.4453564 2.3280735
 1.9009233 4.130048

Validation Epoch 10/10:   0%|          | 0/1 [00:00<?, ?it/s]

Scene: scene_110
Labels: 
 [0.59286314 3.321368   3.9872289  3.7473733  3.4749126  4.903662
 4.6076612  1.6477798  1.4583116  1.898902   1.6272713  4.8462505
 1.4568348  3.3956892  2.0783694  1.454493   2.1345558  2.337087
 2.4154673  4.7286296  4.644076   4.6215067  3.1166477  4.2819877
 3.5449996  4.6295996  4.571115   3.746971   4.557249   3.7359092
 3.1555998  4.8359694  1.5050168  2.0583696  3.202555   3.9031243 ]
Scene: scene_173
Labels: 
 [0.4074984 1.7756864 4.3685474 2.6684132 3.7790463 1.6306722 1.9306862
 2.971937  2.9212248 1.8476715 2.7238503 1.7631072 2.2440672 1.3521923
 3.6121807 1.8163724 1.8638592 2.1998413 4.2758904 1.7675308 2.4085472
 2.049207  1.1624063 2.472029  4.083626  1.2338647 2.2577035 1.2989473
 2.195868  2.8418667 1.6638728 4.673577  2.774746  1.4322546 1.2344766
 3.7412853]
Scene: scene_195
Labels: 
 [0.4346917 4.0873685 4.1074567 4.0546565 4.668514  4.1278005 4.199916
 4.1992664 3.9021645 1.6387109 1.9749628 1.9428326 3.757982  3.851896
 2.0886908 2.721

Epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
Train/Loss,█▇▆▄▂▂▂▁▁▁
Validation/Loss,███▇▆▄▃▃▂▁
Epoch,10
Train/Loss,1.66784
Validation/Loss,2.5377


In [10]:
wandb.finish()

Next, you can check out the following notebook to learn how to run a hyperparameter sweep on our PointNet++ trainig loop using Weights & Biases:

|Tune Hyperparameters using Weights & Biases Sweep|[![](https://colab.research.google.com/assets/colab-badge.svg)](http://wandb.me/pyg-pointnet2-sweep)|